[View in Colaboratory](https://colab.research.google.com/github/akxkcd/fattails/blob/master/rnn_learn.ipynb)

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
num_classes = 2
input_size = 1  # one-hot size
hidden_size = 2  # output from the RNN. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 1  # One by one
num_layers = 1  # one-layer rnn

class Model(nn.Module):
  
  def __init__(self):
    super(Model, self).__init__()
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
    
  def forward(self, hidden, x):
    x = x.view(batch_size, sequence_length, input_size)

    # Propagate input through RNN
    # Input: (batch, seq_len, input_size)
    # hidden: (num_layers * num_directions, batch, hidden_size)
    out, hidden = self.rnn(x, hidden)
    return hidden, out.view(-1, num_classes)
  
  def init_hidden(self):
    # Initialize hidden and cell states
    # (num_layers * num_directions, batch, hidden_size)
    return Variable(torch.zeros(num_layers, batch_size, hidden_size))
  

In [0]:
model = Model()
print(model)

Model(
  (rnn): RNN(1, 2, batch_first=True)
)


In [0]:
one_hot_lookup = [[1],  # 0
                  [0],  # 1
                  [0],  # 2
                  [0],  # 3
                  [0]]  # 4
id2char = ['h', 'i', 'e', 'l', 'o']

x_data = [0, 1, 0, 1, 0, 0]
# y_data = [1, 0, 1, 0, 1, 1]
y_data = [0, 0, 1, 0, 1, 0]

x_one_hot = [one_hot_lookup[i] for i in x_data]
print("x_one_hot:", x_one_hot)

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
  optimizer.zero_grad()
  loss = 0
  hidden = model.init_hidden()

  # print("predicted string: ")
  for input, label in zip(inputs, labels):
    # print(input.size(), label.size())
    hidden, output = model(hidden, input)
    val, idx = output.max(1)
    # print(id2char[idx.data[0]])
    loss += criterion(output, label)
    if epoch == 99:
      print("{} output: {} label:{}".format(input.data[0], idx.data[0], label.data[0]))
  # print(", epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))

  loss.backward()
  optimizer.step()

print("Learning finished!")

x_one_hot: [[1], [0], [1], [0], [1], [1]]
1.0 output: 0 label:0
0.0 output: 0 label:0
1.0 output: 1 label:1
0.0 output: 0 label:0
1.0 output: 1 label:1
1.0 output: 0 label:0
Learning finished!
